# Chapitre 1 - Récupération des données pour l'étude départementale

In [ ]:
# Paramètre(s) du notebook

# VERBOSE=True
VERBOSE=False

OPTIONS=""
if not VERBOSE:
    OPTIONS="--quiet"


## 1- Importation des librairies externes et classiques

In [ ]:
!pip install openpyxl $OPTIONS

In [ ]:
# Importation des librairies 
import requests
from zipfile import ZipFile, Path
import io

import numpy as np
import pandas as pd

In [ ]:
print("1 - Importation des librairies externes et classiques  ====> OK ")

## 2- Téléchargement des données de SIRENE

In [ ]:
### Téléchargement des données Sirene sur les établissements Siret de boucherie-charcuterie (code 4722Z) à l'instant T
# Téléchargement via Insee sirene data  by sirene.fr website

session = requests.session()
URL_ACCUEIL = "https://www.sirene.fr/sirene/public/creation-fichier"
URL_REQUETE_POST = "https://www.sirene.fr/sirene/public/verifier-creation-fichier.action"
URL_DEVIS="https://www.sirene.fr/sirene/client//sirene/client/devis-demande.action"

URL_RECAP="https://www.sirene.fr/sirene/client/telechargement-pret-fichier.action"
URL_TELECHARGEMENT_POST="https://www.sirene.fr/sirene/client/telechargement-fichier"
POST_DATA={
'fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.ferme': 'true',
'fichier.caracteristiques.typeEtablissement': '1',
'fichier.caracteristiques.geoFile': 'false',
'fichier.caracteristiques.cjFile': 'false',
'fichier.caracteristiques.apeFile': 'false',
'fichier.caracteristiques.sirenSiretFile': 'false',
'fichiersCriteres.criteresGeographie.upload': '(binaire)',
'__multiselect_fichier.criteres.geographie.region.modalites': '',
'__multiselect_fichier.criteres.geographie.departement.modalites': '',
'fichiersCriteres.criteresActivite.upload': '(binaire)',
'__multiselect_fichier.criteres.activite.section.modalites': '',
'__multiselect_fichier.criteres.activite.division.modalites': '',
'__multiselect_fichier.criteres.activite.groupe.modalites': '',
'fichier.criteres.activite.sousClasse.modalites': '4722Z',
'__multiselect_fichier.criteres.activite.sousClasse.modalites': '',
'fichiersCriteres.criteresNature.upload': '(binaire)',
'__multiselect_fichier.criteres.nature.entrepriseNiveau1.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau2.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau3.modalites': '',
'__multiselect_fichier.criteres.trancheEffectif.modalites': '',
'fichier.criteres.dateCreation.date': '',
'fichier.criteres.dateCreation.dateFin': '',
'fichier.criteres.dateMaj.date': '',
'fichier.criteres.dateMaj.dateFin': '',
'fichiersCriteres.criteresSirenSiret.upload': '(binaire)',
'__multiselect_fichier.criteres.criteresCatEnt.modalites': '',
'strutsLocale': 'fr_FR',
}

result=session.get(URL_ACCUEIL)
result=session.post(URL_REQUETE_POST, data=POST_DATA)


result=session.get(URL_RECAP)

result=session.get(URL_DEVIS)

result=session.post(URL_TELECHARGEMENT_POST)

donnees_sirene_fr=pd.read_csv(io.StringIO(result.content.decode('utf-8')),dtype=str)


In [ ]:
if VERBOSE:
    display(donnees_sirene_fr.shape)
    display(donnees_sirene_fr.sample(10))

* les données de Sirene.fr sont des données d'entreprises France entière. Pour l'étude, le nombre d'établissement par département est considéré :

In [ ]:
if VERBOSE:
    pd.set_option('display.max_columns', None)
    display(donnees_sirene_fr.sample(10))

In [ ]:
donnees_sirene_fr["departement"]=donnees_sirene_fr.codeCommuneEtablissement.str[:2]

donnees_sirene_fr["departement"]=np.where(donnees_sirene_fr.codeCommuneEtablissement.str[:2]=='97', donnees_sirene_fr.codeCommuneEtablissement.str[:3],donnees_sirene_fr.codeCommuneEtablissement.str[:2])
donnees_sirene_fr_agregees_par_departement = donnees_sirene_fr.groupby("departement").size().reset_index(name='NbBoucheriesParDep').set_index("departement")
if VERBOSE:
    donnees_sirene_fr_agregees_par_departement.describe()

In [ ]:
if VERBOSE:
    donnees_sirene_fr_agregees_par_departement

In [ ]:
if not donnees_sirene_fr_agregees_par_departement.empty:
    print("2 - Téléchargement des données de SIRENE  ====> OK ")
else:
    print("2 - Téléchargement des données de SIRENE  ====> KO ")

## 3- Téléchargement des données du ministère de l'intérieur

In [ ]:
#Téléchargement des données de crimes et délits rapportés déclarés auprès des commissariats 
url_datagouv_min_interieur="https://www.data.gouv.fr/fr/datasets/r/d792092f-b1f7-4180-a367-d043200c1520"

donnees_police_nationale_2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services PN 2018', skiprows=[1,2])
donnees_gendarmerie_nationale_2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services GN 2018', skiprows=[1,2])

del donnees_police_nationale_2018[donnees_police_nationale_2018.columns[0]]
del donnees_gendarmerie_nationale_2018[donnees_gendarmerie_nationale_2018.columns[0]]

In [ ]:
donnees_gendarmerie_nationale_2018[donnees_gendarmerie_nationale_2018.columns[1:]]=donnees_gendarmerie_nationale_2018[donnees_gendarmerie_nationale_2018.columns[1:]].fillna(0).astype(int)
if VERBOSE:
    display(donnees_gendarmerie_nationale_2018)

## Traitement des données de la Police Nationale

In [ ]:
donnees_police_nationale_2018_par_dep=donnees_police_nationale_2018.T
donnees_police_nationale_2018_par_dep.columns = donnees_police_nationale_2018_par_dep.iloc[0]
donnees_police_nationale_2018_par_dep=donnees_police_nationale_2018_par_dep.iloc[1: , :]

donnees_police_nationale_2018_par_dep.drop(donnees_police_nationale_2018_par_dep[donnees_police_nationale_2018_par_dep.index.str[:2]=='98'].index,inplace=True)
donnees_police_nationale_2018_par_dep["departement"]=np.where(donnees_police_nationale_2018_par_dep.index.str[:2]=='97', donnees_police_nationale_2018_par_dep.index.str[:3],donnees_police_nationale_2018_par_dep.index.str[:2])

donnees_police_nationale_2018_par_dep = donnees_police_nationale_2018_par_dep.groupby("departement").sum() 

if VERBOSE:
    display(donnees_police_nationale_2018_par_dep)

* Agrégation par département, transposition et fusion :

In [ ]:
donnees_police_nationale_2018_par_dep = donnees_police_nationale_2018_par_dep.sum(axis=1).reset_index("departement").set_index("departement")
donnees_police_nationale_2018_par_dep.rename(columns={0: "Total des crimes et délits relevés par la Police Nationale"},inplace=True)

## Traitement des données de la Gendarmerie Nationale

In [ ]:
donnees_gendarmerie_nationale_2018_par_dep=donnees_gendarmerie_nationale_2018.T
donnees_gendarmerie_nationale_2018_par_dep.columns = donnees_gendarmerie_nationale_2018_par_dep.iloc[0]
donnees_gendarmerie_nationale_2018_par_dep=donnees_gendarmerie_nationale_2018_par_dep.iloc[1: , :]

donnees_gendarmerie_nationale_2018_par_dep.drop(donnees_gendarmerie_nationale_2018_par_dep[donnees_gendarmerie_nationale_2018_par_dep.index.str[:2]=='98'].index,inplace=True)
donnees_gendarmerie_nationale_2018_par_dep["departement"]=np.where(donnees_gendarmerie_nationale_2018_par_dep.index.str[:2]=='97', donnees_gendarmerie_nationale_2018_par_dep.index.str[:3],donnees_gendarmerie_nationale_2018_par_dep.index.str[:2])

donnees_gendarmerie_nationale_2018_par_dep = donnees_gendarmerie_nationale_2018_par_dep.groupby("departement").sum()

if VERBOSE:
    display(donnees_gendarmerie_nationale_2018_par_dep)

In [ ]:
donnees_gendarmerie_nationale_2018_par_dep = donnees_gendarmerie_nationale_2018_par_dep.sum(axis=1).reset_index("departement").set_index("departement")
donnees_gendarmerie_nationale_2018_par_dep.rename(columns={0: "Total des crimes et délits relevés par la Gendarmerie Nationale"},inplace=True)

if VERBOSE:
    display(donnees_gendarmerie_nationale_2018_par_dep)

## Fusion des données de la Police Nationale et de la Gendarmerie Nationale 

In [ ]:
merged_crimes_delits_2018_par_dep = donnees_police_nationale_2018_par_dep.merge(donnees_gendarmerie_nationale_2018_par_dep, on="departement", how='left')

if VERBOSE:
    display(merged_crimes_delits_2018_par_dep)

In [ ]:
if not merged_crimes_delits_2018_par_dep.empty:
    print("3 - Téléchargement des données du ministère de l'intérieur  ====> OK ")
else:
    print("3 - Téléchargement des données du ministère de l'intérieur  ====> KO ")

## 4 - Téléchargement des revenus localisés

In [ ]:
#Téléchargement des données Filosofi au niveau IRIS (millénisme 2018)
URL_FILO_DECL_2018="https://www.insee.fr/fr/statistiques/fichier/5055909/BASE_TD_FILO_DEC_IRIS_2018.xlsx"
URL_FILO_DISP_2018="https://www.insee.fr/fr/statistiques/fichier/5055909/BASE_TD_FILO_DISP_IRIS_2018.xlsx"

donnees_filo_decl_2018=pd.read_excel(URL_FILO_DECL_2018, sheet_name='IRIS_DEC', skiprows=5)
donnees_filo_disp_2018=pd.read_excel(URL_FILO_DISP_2018, sheet_name='IRIS_DISP', skiprows=5)

In [ ]:
URL_FILO_ZIP_2018="https://www.insee.fr/fr/statistiques/fichier/5009236/base-cc-filosofi-2018_CSV_geo2021.zip"
#Télécharger le zip de l'URL
r=requests.get(URL_FILO_ZIP_2018)

open("base-cc-filosofi-2018_CSV_geo2021.zip", 'wb').write(r.content)
    
    
with ZipFile("base-cc-filosofi-2018_CSV_geo2021.zip",'r') as myzip:
    data = myzip.open("cc_filosofi_2018_COM-geo2021.CSV")
    data2 = myzip.open("cc_filosofi_2018_DEP-geo2021.CSV")
    
donnees_filo_communes_2018=pd.read_csv(data,sep=';')    
donnees_filo_departement_2018=pd.read_csv(data2,sep=';')    

In [ ]:
if VERBOSE:
    display(donnees_filo_communes_2018.head(10))
    display(donnees_filo_departement_2018)

In [ ]:
if VERBOSE:
    display(donnees_filo_departement_2018.describe())

In [ ]:
donnees_filo_departement_2018=donnees_filo_departement_2018.set_index("CODGEO")

In [ ]:
if VERBOSE:
    display(donnees_filo_departement_2018)

In [ ]:
if not donnees_filo_departement_2018.empty:
    print("4 - Téléchargement des revenus localisés  ====> OK ")
else:
    print("4 - Téléchargement des revenus localisés  ====> KO ")

## 5 - Téléchargement des populations légales

In [ ]:
#Téléchargement des populations légales du millénisme 2018 
URL_POP_LEG="https://www.insee.fr/fr/statistiques/fichier/4989724/ensemble.zip"

#Télécharger le zip de l'URL
r=requests.get(URL_POP_LEG)

open("ensemble.zip", 'wb').write(r.content)
    
    
with ZipFile("ensemble.zip",'r') as myzip:
    data = myzip.open("Departements.csv")

donnees_pop_leg_18=pd.read_csv(data,sep=';')     

In [ ]:
if VERBOSE:
    display(donnees_pop_leg_18.sample(10))

In [ ]:
donnees_pop_leg_18=donnees_pop_leg_18.set_index("CODDEP")

In [ ]:
if VERBOSE:
    display(donnees_pop_leg_18.describe())

In [ ]:
if not donnees_pop_leg_18.empty:
    print("5 - Téléchargement des populations légales  ====> OK ")
else:
    print("5 - Téléchargement des populations légales  ====> KO ")


## 6 - Téléchargement des données d'emplois localisées

In [ ]:
#Téléchargement des données d'emploi localisées
URL_TAUX_CHOMAGE_PAR_DEP="https://www.insee.fr/fr/statistiques/fichier/2012804/sl_etc_2021T2.xls"


donnees_chomage_par_departement_series_longues=pd.read_excel(URL_TAUX_CHOMAGE_PAR_DEP, sheet_name='Département', skiprows=3)

donnees_chomage_par_departement_series_longues=donnees_chomage_par_departement_series_longues.iloc[:100]
donnees_chomage_par_departement_series_longues=donnees_chomage_par_departement_series_longues.set_index("Code")
donnees_chomage_par_departement_series_longues.index=donnees_chomage_par_departement_series_longues.index.astype(str)

In [ ]:
if VERBOSE:
    display(donnees_chomage_par_departement_series_longues.describe())

In [ ]:
if not donnees_chomage_par_departement_series_longues.empty:
    print("6 - Téléchargement des données d'emplois localisées  ====> OK ")
else:
    print("6 - Téléchargement des données d'emplois localisées  ====> KO ")


## 7- Téléchargement des données de répartition de gendarmes et policiers en 2019 sur le territoire (au niveau départemental)

In [ ]:
URL_NB_PN_GN_DEP = "https://www.insee.fr/fr/statistiques/fichier/5763601/IREF_SECUR21-F29.xlsx"
donnees_nb_pn_gn_dep=pd.read_excel(URL_NB_PN_GN_DEP, sheet_name='Figure 3', skiprows=1)
donnees_nb_pn_gn_dep = donnees_nb_pn_gn_dep.set_index("Unnamed: 0")
donnees_nb_pn_gn_dep=donnees_nb_pn_gn_dep.iloc[:100]
donnees_nb_pn_gn_dep=donnees_nb_pn_gn_dep.rename(columns={"Unnamed: 0": "CODGEO", "Unnamed: 1": "Département", "Taux pour 10 000 habitants":"Nb_PN_GN_dep_10k_hab"}, index={'Unnamed:0': 'CODDEP'})
donnees_nb_pn_gn_dep.index = donnees_nb_pn_gn_dep.index.astype(str)
donnees_nb_pn_gn_dep.index.names = ['CODDEP']
donnees_nb_pn_gn_dep['Nb_PN_GN_dep_100k_hab']=round(donnees_nb_pn_gn_dep['Nb_PN_GN_dep_10k_hab']*10)
donnees_nb_pn_gn_dep.drop(['Nb_PN_GN_dep_10k_hab'], axis = 1, inplace = True)

if VERBOSE:
    display(donnees_nb_pn_gn_dep)


In [ ]:
if not donnees_nb_pn_gn_dep.empty:
    print("7 - Téléchargement des données de répartition de gendarmes et policiers en 2019 sur le territoire (au niveau départemental)  ====> OK ")
else:
    print("7 - Téléchargement des données de répartition de gendarmes et policiers en 2019 sur le territoire (au niveau départemental)  ====> KO ")


## 8 - Fusion des données

In [ ]:
donnees_2018=merged_crimes_delits_2018_par_dep.merge(donnees_filo_departement_2018,left_index=True,right_index=True)
donnees_2018=donnees_2018.merge(donnees_pop_leg_18,left_index=True,right_index=True)
donnees_2018=donnees_2018.merge(donnees_sirene_fr_agregees_par_departement,left_index=True,right_index=True)
donnees_2018=donnees_2018.merge(donnees_chomage_par_departement_series_longues,left_index=True,right_index=True)
donnees_2018=donnees_2018.merge(donnees_nb_pn_gn_dep,left_index=True,right_index=True)
donnees_2018=donnees_2018[['REG', 'Libellé', 'Total des crimes et délits relevés par la Police Nationale', 'Total des crimes et délits relevés par la Gendarmerie Nationale', 'MED18' , 'TP6018', 'D118', 'D918', 'RD18', 'PTOT','T1_2018', 'NbBoucheriesParDep', 'Nb_PN_GN_dep_100k_hab']]

if VERBOSE:
    display(donnees_2018)

* On se restreint au champ métropolitain car les données d'outre mer ne sont pas exhaustives sur toutes nos sources 

In [ ]:
donnees_2018 = donnees_2018[donnees_2018.index.str[:2]!='97']

if VERBOSE:
    display(donnees_2018)
    display(donnees_2018.describe())

* On fait l'hypothèse d'unicité du dépôt de plainte ou de signalement en gardarmerie ou bien en commissariat, ce qui permet de sommer les crimes et délits au niveau départemental

In [ ]:
donnees_2018['Total des crimes et délits relevés par la PN et GN'] = donnees_2018[['Total des crimes et délits relevés par la Police Nationale', 'Total des crimes et délits relevés par la Gendarmerie Nationale']].sum(axis=1)

In [ ]:
donnees_2018 = donnees_2018.rename(columns={"Total des crimes et délits relevés par la Police Nationale": "Crim_Del_PN", "Total des crimes et délits relevés par la Gendarmerie Nationale": "Crim_Del_GN", "Total des crimes et délits relevés par la PN et GN": "Crim_Del_PN_GN"})

if VERBOSE:
    display(donnees_2018)

In [ ]:
if not donnees_2018.empty:
    print(" 8 - Fusion des données  ====> OK ")
else:
    print(" 8 - Fusion des données  ====> KO ")

## 9 - On créé une base de données rapportant le nombre de crimes et délits, et le nombre de boucheries pour 100 000 habitants 


In [ ]:
donnees_2018_hab = donnees_2018
donnees_2018_hab['Nb_Boucherie_dep_hab']=round(donnees_2018_hab['NbBoucheriesParDep']/donnees_2018_hab['PTOT']*100000)
donnees_2018_hab['Crim_Del_GN_hab']=round(donnees_2018_hab['Crim_Del_GN']/donnees_2018_hab['PTOT']*100000)
donnees_2018_hab['Crim_Del_PN_hab']=round(donnees_2018_hab['Crim_Del_PN']/donnees_2018_hab['PTOT']*100000)
donnees_2018_hab['Crim_Del_PN_GN_hab']=round(donnees_2018_hab['Crim_Del_PN_GN']/donnees_2018_hab['PTOT']*100000)
donnees_2018_hab.drop(['Crim_Del_PN', 'Crim_Del_GN', 'PTOT', 'NbBoucheriesParDep'], axis=1, inplace = True)

if VERBOSE:
    display(donnees_2018_hab)


In [ ]:
%store donnees_2018
%store donnees_2018_hab

In [ ]:
if not donnees_2018_hab.empty:
    print(" 9 - Export des données  ====> OK ")
else:
    print(" 9 - Export des données  ====> OK ")